In [4]:
import numpy as np
from scipy.stats import gmean
import cvxopt as opt
from cvxopt import matrix, spmatrix, sparse
from cvxopt.solvers import qp, options
from cvxopt import blas

C:\Users\samga\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Creating an MVO

In [5]:
def geo_mean(returns):
    geo = []
    n = len(returns.transpose())
    for i in range(0,n):
        geo = geo + [np.exp(np.log(rets[:,i]+1).mean())-1]
    return geo

In [6]:
def quad_opt_func(Q,n):
    
    return 2*Q

In [7]:
def lin_opt_func(r,n):
    if r == False:
        return np.zeros([n,1])
    else:
        return r

In [15]:
def inequality_constraints(n):
    
    # Inequality Constraint
    # Expected Return Over 0.0035
    G1 = np.matrix([-1])*mu
    h1 = np.matrix(-0.035)

    # Lower Bound on Each Element!
    G2 = -1*np.identity(n)
    h2 = np.zeros([n,1])

    #Concat all Answers
    G = np.concatenate((G1,G2),axis=0)
    h = np.concatenate((h1,h2),axis=0)
    
    return G,h

In [16]:
def equality_constraints(n):
    
    # Equality Constraint
    # Weight sum is 1
    A1 = np.ones([1,n])
    b1 = np.ones([1,1])
    
    
    #Concat All Equality
    A = A1#np.concatenate((A1),axis=0)
    b = b1#np.concatenate((b1),axis=0)
    
    return A,b
    

In [17]:
def MVO(mu,Q,x0):
    
    #NOTE: X0 is not used yet but it will be for transaction costs
    
    # Number of Assets
    n = len(Q)
    
    # ----- Constraints -----------
    
    # Equality Constraint
    
    A,b = equality_constraints(n)
    
    # Inequality Constraint
    
    G,h = inequality_constraints(n)
    
    
    # --- Quadtratic Optimization Function --------
    #quad = 2*Q
    quad = quad_opt_func(Q,n)
    # ------ Linear Optimization Function ---------
    #r = np.zeros([n,1])
    r = lin_opt_func(False,n)
    
    
    # ------------ Optimize! --------
    sol = qp(matrix(quad), matrix(r), matrix(G), matrix(h), matrix(A), matrix(b))['x']
    
    
    return sol
    
    

# Inputs

# Set Up

https://towardsdatascience.com/quadratic-optimization-with-constraints-in-python-using-cvxopt-fc924054a9fc

##### Initial Setup
- n is number of stocks

- Q is quadtratic part of min function nxn matrix

- r is linear min function nx1 matrix

##### EQUALITY CONSTRAINTS

##### A*x = b
- A is m x n matrix with m constraints

- b is m x 1 matrix with m constraints

##### INEQUALITY CONSTRAINTS

##### G*x = h

- G is m x n matrix with m constraints

- b is m x 1 matrix with m constraints


## Set Up Universe of Returns and Volatility

In [22]:
prices = np.matrix([[120, 54.5, 73.8],[129.6,45.78,70.85],[152.93,48.07,72.98],[144.52,50.57,76.05],[14005.52,53.57,78.05]])
rets = prices[1:,:]/prices[0:-1,:] -1
mu = geo_mean(rets)
Q = np.cov(rets.transpose())
x0 = False

In [23]:
MVO(mu,Q,x0)

     pcost       dcost       gap    pres   dres
 0:  4.5860e-03 -9.5258e-01  6e+00  2e+00  5e+00
 1:  6.7297e-03 -4.2536e-01  6e-01  8e-02  2e-01
 2:  1.4194e-01  1.2865e-01  2e-01  2e-02  5e-02
 3:  1.9262e-01  1.8797e-01  2e-02  1e-03  3e-03
 4:  2.0230e-01  2.0225e-01  2e-04  1e-05  3e-05
 5:  2.0240e-01  2.0240e-01  2e-06  1e-07  3e-07
 6:  2.0240e-01  2.0240e-01  2e-08  1e-09  3e-09
Optimal solution found.


<3x1 matrix, tc='d'>

# Setting UP THe FUnction Above

## Set Up Equality Constraints (Ax=b)

In [13]:
# Weight sum is 1
A1 = np.ones([1,3])
b1 = np.ones([1,1])


## Setup Inequality Constraints (G*x<h)

In [14]:
# Expected Return Over 0.0035
G1 = np.matrix([-1])*mu
h1 = np.matrix(-0.035)

# Lower Bound on Each Element!
G2 = -1*np.identity(n)
h2 = np.zeros([n,1])

#Concat all Answers
G = np.concatenate((G1,G2),axis=0)
h = np.concatenate((h1,h2),axis=0)

NameError: name 'n' is not defined

## Set Up Quadratic Optimization

In [533]:
quad = 2*Q

## Set Up Linear Optimization

In [534]:
r = np.zeros([3,1])

## Solve!

In [556]:
sol = qp(matrix(quad), matrix(r), matrix(G), matrix(h), matrix(A), matrix(b))['x']

     pcost       dcost       gap    pres   dres
 0:  4.0094e-03 -9.7501e-01  5e+00  2e+00  2e+00
 1:  3.9349e-03 -5.5588e-01  6e-01  5e-02  6e-02
 2:  4.6695e-03 -1.1841e-01  1e-01  1e-02  1e-02
 3:  5.3961e-03  6.3227e-04  5e-03  1e-16  6e-17
 4:  3.3422e-03  2.7664e-03  6e-04  2e-16  1e-17
 5:  2.9803e-03  2.9734e-03  7e-06  2e-16  9e-18
 6:  2.9758e-03  2.9757e-03  7e-08  1e-16  2e-17
Optimal solution found.


In [557]:
print(sol)

[ 4.63e-01]
[ 3.00e-06]
[ 5.37e-01]



In [450]:
print(hh)

[-3.50e-02]
[ 0.00e+00]

